In [1]:
import os
import numpy as np
import sys
import json
os.chdir(os.path.join(os.getcwd(), '../..'))
from src.download_models import get_all_model_dict

In [6]:
uncertainty_files = ["verbalised_and_sampling_hybrid_uncertainty.npy", "sample_avg_dev_uncertainty.npy", "verbalised_uncertainty.npy"]

model_uqs = {}
models = get_all_model_dict()
for model_name in models.keys():
    model_results_dir = os.path.join("results", model_name)
    uqs = {}
    for uncertainty_file in uncertainty_files:
        uncertainty_path = os.path.join(model_results_dir, "uncertainty_estimates", uncertainty_file)
        uncertainty_estimates = np.load(uncertainty_path)
        uqs[uncertainty_file] = uncertainty_estimates
        
    model_uqs[model_name] = uqs

In [11]:
for model_name, uqs_dict in model_uqs.items():
    print(f"==============Model: {model_name}===================")
    for uq_type, uqs in uqs_dict.items():
        valid_uqs = uqs[uqs != -1]
        print(f"-----------UQ type: {uq_type}-----------")
        print(f"Min: {np.min(valid_uqs)}")
        print(f"Max: {np.max(valid_uqs)}")
        print(f"Mean: {np.mean(valid_uqs)}")
        print(f"Median: {np.median(valid_uqs)}")
        print(f"Std: {np.std(valid_uqs)}")
        print(f"Variance: {np.var(valid_uqs)}")
        print(f"Full Length with invalid scores: {len(uqs)}")
        print(f"Invalid Scores where -1 is value: {np.sum(uqs == -1)}")
        print(f"Lower score than 0.5: {np.sum(uqs < 0.5)}")

==============Model: openai===================
-----------UQ type: verbalised_and_sampling_hybrid_uncertainty.npy-----------
Min: 0.06448051948051949
Max: 0.9212337662337662
Mean: 0.539340909090909
Median: 0.6683116883116883
Std: 0.32442170280784355
Variance: 0.10524944125274077
Full Length with invalid scores: 100
Invalid Scores where -1 is value: 0
Lower score than 0.5: 38
-----------UQ type: sample_avg_dev_uncertainty.npy-----------
Min: 0.0
Max: 1.0
Mean: 0.42090909090909095
Median: 0.35064935064935066
Std: 0.2758882678060769
Variance: 0.07611433631303761
Full Length with invalid scores: 100
Invalid Scores where -1 is value: 0
Lower score than 0.5: 64
-----------UQ type: verbalised_uncertainty.npy-----------
Min: 0.05
Max: 0.95
Mean: 0.5525
Median: 0.7
Std: 0.3717778234376009
Variance: 0.13821874999999997
Full Length with invalid scores: 100
Invalid Scores where -1 is value: 0
Lower score than 0.5: 38
==============Model: Meta-Llama-3.1-8B-Instruct-GPTQ-INT4===================
----